# Currency Conversion using Tools with LLM

Create a workflow such that a LLM can use an exchange rate tool to convert one currency to another.

In [ ]:
# Tool Creation

from langchain_core.tools import InjectedToolArg
from typing import Annotated
from langchain.tools import tool
from dotenv import load_dotenv
import os
import requests
load_dotenv()

@tool
def get_exchange_rate(base_currency:str, target_currency:str)->float:
    """This function fetches the conversion factor between the base currency and the target currency"""
    api_key=os.getenv('EXCHANGERATE_API_KEY')
    url=f'https://v6.exchangerate-api.com/v6/{api_key}/pair/{base_currency}/{target_currency}'
    response=requests.get(url)
    data=response.json()
    return data['conversion_rate']

@tool
def convert_currency(base_currency_val:float, conversion_rate: Annotated[float,InjectedToolArg])->float: # what this code means is: Hey LLM do not set the value of this argument, I (developer) will inject the value of this argument after running earlier tool calls
    """This function is used to convert base currency to target currency given the base currency value and conversion rate """
    return base_currency_val*conversion_rate


In [ ]:
conversion_rate=get_exchange_rate.invoke({'base_currency':'USD','target_currency':'INR'})
print(conversion_rate)
print(convert_currency.invoke({'base_currency_val':12,'conversion_rate':conversion_rate}))

In [ ]:
# Tool Binding

from langchain_groq import ChatGroq
llm=ChatGroq(model='llama-3.3-70b-versatile',temperature=0)
llm_with_tools=llm.bind_tools([get_exchange_rate,convert_currency])

In [ ]:
# Tool Execution

from langchain.messages import HumanMessage
messages=[HumanMessage('What is the exchange rate between USD to INR and based on that convert 53.46 USD to INR')]

In [ ]:
ai_msg=llm_with_tools.invoke(messages)
messages.append(ai_msg)
ai_msg.tool_calls
# here you can see the LLM made a mistake, it tackled both questions simultaneously but since it handt recieved value of conversion rate from the first tool, it hallucinated and gave the value it last remembered.
# to avoid this we use Injected Tool Argument

In [ ]:

for tool_call in ai_msg.tool_calls:
    # execute the first tool and get value of conversion rate
    if tool_call['name']=='get_exchange_rate':
        tool1_msg=get_exchange_rate.invoke(tool_call)
        rate=float(tool1_msg.content)
        print(rate)
        messages.append(tool1_msg)
    # exceute the second tool and pass value of conversion rate
    if tool_call['name']=='convert_currency':
        tool_call['args']['conversion_rate']=rate
        tool_call['args']['base_currency_val']=float(tool_call['args']['base_currency_val'])
        tool2_msg=convert_currency.invoke(tool_call)
        messages.append(tool2_msg)

print(messages)

In [ ]:
result=llm_with_tools.invoke(messages).content
print(result)